In [123]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:90%;}
div.CodeMirror {font-family:Consolas; font-size:18pt;}
div.output {font-size:19pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:17pt;}
</style>
"""))

<b><font color="red" size="6">ch10. 데이터 베이스 연동</font></b>

# 1절. SQLite 데이터 베이스 연결
- SQLite 데이터 베이스는 별도의 DBMS없이 SQL을 이용하여 DB 엑세스 할 수 있도록 만든 간단한 디스크 기반의 DB 제공
- C 라이브러리
- SQLite는 프로토타입을 만들 때 사용
- 프로젝트 단계 : 분석 → 설계 → 구현 → 테스트 → 고객에게 배포 → 유지보수
-               프로토타입(SQLite)  시제품     완제품(Oracle, MySql, Maria, ...)
-
- [DB browser for SQLite](https://sqlitebrowser.org/dl/)
## 1.1 SQLite broswer 설치 및 sqlite3 패키지 load

In [5]:
import sqlite3
sqlite3.sqlite_version # sqlite3 패키지의 버전

'3.40.1'

In [6]:
import pandas as pd
pd.__version__

'1.5.3'

In [7]:
import numpy as np
np.__version__

'1.23.5'

## 1.2 데이터베이스 연결
- SQLite로 DB 연결시, DB파일이 있으면 연결, DB파일이 없으면 빈 DB 파일 생성

In [8]:
# DB 연결
conn = sqlite3.connect('data/ch10_example.db')
conn

In [9]:
# 커서 객체 생성. 커서는 SQL문 실행시키고, 결과데이터를 조회(select, 그외)하는데 사용하는 객체
cursor = conn.cursor()
cursor

In [10]:
cursor.execute('''
    CREATE TABLE MEMBER (
        NAME TEXT,
        AGE  INT,
        EMAIL TEXT
    )
''')

In [11]:
cursor.execute('DROP TABLE MEMBER')

In [12]:
cursor.execute('''
    CREATE TABLE MEMBER (
        NAME TEXT,
        AGE  INT,
        EMAIL TEXT
    )
''')

In [13]:
cursor.execute('INSERT INTO MEMBER VALUES (\'홍길동\', 20, \'h@h.com\')')
print('수행 결과 행수 :', cursor.rowcount)
sql = "INSERT INTO MEMBER VALUES ('신길동', 25, 'S@S.COM')"
cursor.execute(sql)
print('수행 결과 행수 :', cursor.rowcount)
cursor.execute('INSERT INTO MEMBER VALUES (\'신림동\', 30, \'l@h.com\')')
print('수행 결과 행수 :', cursor.rowcount)

수행 결과 행수 : 1
수행 결과 행수 : 1
수행 결과 행수 : 1


In [14]:
conn.commit() # 反. conn.rollback() DML문에서만 commit

In [15]:
# SQL전송 결과는 cursor가 가리킴
cursor.execute("SELECT * FROM MEMBER ORDER BY NAME")

In [16]:
# insert, update, delete 문 실행결과 : cursor.rowcount
# select문 실행 결과를 받는 함수들
    ## fetchone() : 결과를 한행씩 받을 때 (튜플)
    ## fetchall() : 결과를 모두 받을 때 (튜플 list)
    ## fetchmany(n) : 결과를 n행 받을 때 (튜플 list)
print(cursor.fetchall())

[('신길동', 25, 'S@S.COM'), ('신림동', 30, 'l@h.com'), ('홍길동', 20, 'h@h.com')]


In [17]:
print(cursor.fetchall()) # 한번 소요된 cursor 객체는 다시 fetch할 수 없음

[]


In [18]:
cursor.execute("SELECT NAME, AGE, EMAIL FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members # 튜플 리스트

[('홍길동', 20, 'h@h.com'), ('신길동', 25, 'S@S.COM'), ('신림동', 30, 'l@h.com')]

In [20]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('신길동', 25, 'S@S.COM')
('신림동', 30, 'l@h.com')


In [22]:
# 한줄씩 읽기
cursor.execute("SELECT NAME, AGE, EMAIL FROM MEMBER ORDER BY AGE")
member_list = []
while True:
    member = cursor.fetchone() # SQL문 수행 결과 한줄 가져오기
    if member is None:
        break
    #print(member)
    member_list.append({'name':member[0], 'age': member[1], 'email': member[2]})

In [23]:
member_list

[{'name': '홍길동', 'age': 20, 'email': 'h@h.com'},
 {'name': '신길동', 'age': 25, 'email': 'S@S.COM'},
 {'name': '신림동', 'age': 30, 'email': 'l@h.com'}]

In [25]:
# 최상위 n행 읽기
cursor.execute("SELECT NAME, AGE, EMAIL FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(10):
    print(member)

('홍길동', 20, 'h@h.com')
('신길동', 25, 'S@S.COM')
('신림동', 30, 'l@h.com')


In [26]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age  = age
        self.email= email
    def as_dic(self): # insert문 전송시 필요
        return {'name':self.name,
                'age':self.age,
                'email':self.email}
    def __str__(self):
        return "{}\t{}\t{}".format(self.name, self.age, self.email)
def to_member(*row): # 튜플 데이터를 매개변수로 받아 Member형 객체로 return
    return Member(row[0], row[1], row[2])

In [27]:
dbreadmember = ('홍길동', 20, 'h@h.com')
m = to_member(*dbreadmember) # 튜플 언패킹
print(m)
print(m.as_dic())

홍길동	20	h@h.com
{'name': '홍길동', 'age': 20, 'email': 'h@h.com'}


In [29]:
# DB 검색 결과를 객체 list로 
cursor.execute('SELECT NAME, AGE, EMAIL FROM MEMBER')
member_list = [] # sql문 수행 결과를 담을 객체 list
members = cursor.fetchall() # 튜플 list
#print(members)
for member in members:
    member_list.append(to_member(*member))

In [30]:
type(members[0]), type(member_list[0])

(tuple, __main__.Member)

In [31]:
for member in member_list:
    print(member)

홍길동	20	h@h.com
신길동	25	S@S.COM
신림동	30	l@h.com


In [32]:
cursor.close() # cursor.close()는 생략 가능
conn.close()

## 1.3 SQL구문에 파라미터 사용하기
- qmark(DB에 따라 불가한 경우가 있음)
- named(추천)

In [33]:
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동', '신길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('신길동', 25, 'S@S.COM')]

In [36]:
# 파라미터 사용하기 : qmark 방법 이용
name1 = input('검색할 이름1 :')
name2 = input('검색할 이름2 :')
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", (name1, name2))
cursor.fetchall()

검색할 이름1 :홍길동
검색할 이름2 :신길동


[('홍길동', 20, 'h@h.com'), ('신길동', 25, 'S@S.COM')]

In [38]:
# 파라미터 사용하기 : qmark 방법 이용
name1 = input('검색할 이름1 :')
name2 = input('검색할 이름2 :')
names = (name1, name2)
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", names)
cursor.fetchall()

검색할 이름1 :홍길동
검색할 이름2 :신길동


[('홍길동', 20, 'h@h.com'), ('신길동', 25, 'S@S.COM')]

In [41]:
# 파라미터 사용하기 : named 방법 이용
name = input('검색할 이름은 > ')
cursor.execute("SELECT * FROM MEMBER WHERE NAME=:name", {'name':name})
members = cursor.fetchall()
if len(members):
    print(members)
else : 
    print('해당 이름의 데이터가 없습니다', members)

검색할 이름은 > 홍홍홍
해당 이름의 데이터가 없습니다 []


In [43]:
# 파라미터 사용하기 :  named 방법 이용
name1 = input('검색할 이름1 :')
name2 = input('검색할 이름2 :')
names = {'name1':name, 'name2':name2}
# cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1, :name2)", {'name1':name1,
#                                                                        'name2':name2})
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1, :name2)", names)
cursor.fetchall()

검색할 이름1 :홍홍홍
검색할 이름2 :홍길동


[('홍길동', 20, 'h@h.com')]

점심먹고 컴퓨터 켬

In [3]:
import sqlite3
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()

In [4]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age  = age
        self.email= email
    def as_dic(self): # insert문 전송시 필요
        return {'name':self.name,
                'age':self.age,
                'email':self.email}
    def __str__(self):
        return "{}\t{}\t{}".format(self.name, self.age, self.email)
def to_member(*row): # 튜플 데이터를 매개변수로 받아 Member형 객체로 return
    return Member(row[0], row[1], row[2])

In [8]:
# MEMBER 테이블에 입력(사용자로부터 이름, 나이, 메일을 입력받아 insert)
try:
    name = input('입력할 이름은 : ')
    age = int(input('입력할 나이는 : '))
except ValueError:
    print('유효하지 않은 나이를 입력하신 경우 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
inputdata = {'name':name, 'age':age, 'email':email} # named 방식
inputdata2 = (name, age, email) # qmark 방식
# cursor.execute('INSERT INTO MEMBER VALUES (?, ?, ?)', inputdata2)
cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)', inputdata)
conn.commit()
if cursor.rowcount == 1:
    print('저장완료')

입력할 이름은 : 임길동
입력할 나이는 : 33
입력할 메일은 :l@l.com
저장완료


In [11]:
# MEMBER 테이블에 입력(사용자로부터 이름, 나이, 메일을 입력받아 insert)
try:
    name = input('입력할 이름은 : ')
    age = int(input('입력할 나이는 : '))
except ValueError:
    print('유효하지 않은 나이를 입력하신 경우 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')

newMember = Member(name, age, email)
# print(newMember.as_dic())
cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)', newMember.as_dic())
conn.commit()
if cursor.rowcount==1:
    print('입력 성공')

입력할 이름은 : 김수한
입력할 나이는 : 12
입력할 메일은 :k@a.com
{'name': '김수한', 'age': 12, 'email': 'k@a.com'}
입력 성공


In [13]:
# member 테이블에 데이터를 n번 입력하고 
# 이름에 0을 입력할 때까지 이름, 나이, 메일을 받아 insert
# 이름에 0을 입력하면 이때까지 입력한 데이터들을 출력하고 몇명을 입력했는지도 출력

member_list = [] # 입력한 member들이 저장될 list변수(딕셔너리 리스트)
while True:
    try:
        name = input('입력할 이름은(종료는 0) : ')
        if name=='0':
            break
        age = int(input('입력할 나이는 : '))
        email = input('입력할 메일은 : ')
    except ValueError:
        print('유효하지 않은 나이를 입력할 경우 18세로 초기화')
        age = 18
        email = input('입력할 메일은 : ')
    newMember = {'name':name, 'age':age, 'email':email}
    member_list.append(newMember)
    cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)', newMember)
    
conn.commit()
print('입력된 데이터는 ', len(member_list), '명입니다', sep='')
for member in member_list:
    print("{}\t{}\t{}".format(member['name'], member['age'], member['email']))

입력할 이름은(종료는 0) : 간간간
입력할 나이는 : 10
입력할 메일은 : ka
입력할 이름은(종료는 0) : 난난난
입력할 나이는 : 20
입력할 메일은 : na
입력할 이름은(종료는 0) : 0
입력된 데이터는 2명입니다
간간간	10	ka
난난난	20	na


In [16]:
member_list = [] # 입력한 member들이 저장될 list변수(객체 리스트)
while True:
    try:
        name = input('입력할 이름은(종료는 0) : ')
        if name=='0':
            break
        age = int(input('입력할 나이는 : '))
        email = input('입력할 메일은 : ')
    except ValueError:
        print('유효하지 않은 나이를 입력할 경우 18세로 초기화')
        age = 18
        email = input('입력할 메일은 : ')
    newMember = Member(name, age, email)
    member_list.append(newMember)
    cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)', newMember.as_dic())
    
conn.commit()
print('입력한 자료는 다음과 같이 {}명입니다'.format(len(member_list)))
print('이름\t나이\t메일')
for member in member_list:
    print(member)

입력할 이름은(종료는 0) : 이르음
입력할 나이는 : 33
입력할 메일은 : k@l.com
입력할 이름은(종료는 0) : 핸드폰
입력할 나이는 : 22
입력할 메일은 : h@h.com
입력할 이름은(종료는 0) : 0
입력한 자료는 다음과 같이 2명입니다
이름	나이	메일
이르음	33	k@l.com
핸드폰	22	h@h.com


In [17]:
# 입력확인은 DB browser 나 python으로
cursor.execute('SELECT * FROM MEMBER')
cursor.fetchall()

[('홍길동', 20, 'h@h.com'),
 ('신길동', 25, 'S@S.COM'),
 ('신림동', 30, 'l@h.com'),
 ('홍낄뚱', 18, 'h@h.com'),
 ('임길동', 33, 'l@l.com'),
 ('김수한', 12, 'k@a.com'),
 ('가가가', 10, 'ka'),
 ('나나나', 20, 'na'),
 ('간간간', 10, 'ka'),
 ('난난난', 20, 'na'),
 ('홍', 2, 'h@h'),
 ('이르음', 33, 'k@l.com'),
 ('핸드폰', 22, 'h@h.com')]

In [18]:
cursor.close()
conn.close()

# 2절. 오라클 데이터 베이스 연결
- [cx_Oracle](https://cx-oracle.readthedocs.io/en/latest/) 사용

In [20]:
import cx_Oracle

In [22]:
# conn 얻어오는 방법1
oracle_dsn = cx_Oracle.makedsn(host="localhost", port=1521, sid='xe')
print(oracle_dsn)
conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
conn
# 여기서 예외가 날 경우 VC_redist.x64.exe 인스톨

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))


<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [23]:
conn.close()

In [24]:
# conn 얻어오는 방법2
conn = cx_Oracle.connect("scott", "tiger", "localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [25]:
# cursor 객체 생성
cursor = conn.cursor()
sql = "SELECT * FROM EMP"
cursor.execute(sql)
emp = cursor.fetchall()

In [26]:
for e in emp: # 튜플 리스트라 field가 많은 경우 가독성 다소 떨어짐
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [33]:
import pandas as pd
emp_df = pd.DataFrame(emp)
emp_df.head()

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [34]:
# select문 수행한 필드 정보
cursor.description

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [36]:
result = []
for description in cursor.description:
    result.append(description[0])
result

['empno', 'ename', 'job', 'mgr', 'hiredate', 'sal', 'comm', 'deptno']

In [37]:
[row[0] for row in cursor.description]

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [39]:
emp_df.columns

RangeIndex(start=0, stop=8, step=1)

In [42]:
emp_df.columns = [row[0] for row in cursor.description]
emp_df.sample() # 임의의 한줄 출력

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
12,7902,FORD,ANALYST,7566.0,1981-12-03,3000.0,NaN,20


In [44]:
# 검색할 이름을 사용자에게 받아 해당 내용을 출력
ename = input('검색할 이름은 : ')
sql = "SELECT * FROM EMP WHERE ENAME=UPPER(:name)"
cursor.execute(sql, {'name':ename})
emp = cursor.fetchall()
if len(emp):
    for e in emp:
        print(e)
else:
    print('입력하신 이름의 데이터는 없습니다')

검색할 이름은 : scott
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)


In [49]:
fieldnames = [description[0] for description in cursor.description]
print(fieldnames)
print(e)

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)


In [51]:
for idx in range(len(e)):
    print("{}:{}".format(fieldnames[idx], e[idx] if e[idx] is not None else ''))

EMPNO:7788
ENAME:SCOTT
JOB:ANALYST
MGR:7566
HIREDATE:1982-12-09 00:00:00
SAL:3000.0
COMM:
DEPTNO:20


In [52]:
list(zip(fieldnames, e))

[('EMPNO', 7788),
 ('ENAME', 'SCOTT'),
 ('JOB', 'ANALYST'),
 ('MGR', 7566),
 ('HIREDATE', datetime.datetime(1982, 12, 9, 0, 0)),
 ('SAL', 3000.0),
 ('COMM', None),
 ('DEPTNO', 20)]

In [54]:
for fieldname, data in zip(fieldnames, e):
    print("{}:{}".format(fieldname, data if data is not None else '-'))

EMPNO:7788
ENAME:SCOTT
JOB:ANALYST
MGR:7566
HIREDATE:1982-12-09 00:00:00
SAL:3000.0
COMM:-
DEPTNO:20


In [55]:
cursor.close()
conn.close()

# 3절. 연습문제

In [58]:
from IPython.display import display, HTML
display(HTML("""<style>
span.toc-item-num{display:none;}
</style>"""))

## 클래스 생성

In [78]:
class Member:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
        
    def __str__(self):
        return "{:>5}\t{:3}\t{:15}\t{:15}\t{:3}\t{}".format('*'*self.grade, 
                self.name, self.phone, self.email, self.age, self.etc) 

    def as_dict(self): #insert할 때 사용
        return {"name":self.name, "phone":self.phone, 
                "email":self.email, "age":self.age, "grade":self.grade, 
                "etc":self.etc}

def to_member(*row): # 튜플 데이터를 매개변수로 받아 Member형 객체로 반환
    return Member(row[0], row[1], row[2], row[3], row[4], row[5])

## 1번 기능. 입력

In [93]:
def insert_member():
    '사용자로부터 이름, 전화, 이메일, 나이, 등급(1~5), 기타특징 입력받아 DB에 추가한다'
    pass

In [124]:
# 입력 test
import sqlite3
conn = sqlite3.connect('data/ch10_data.db')
insert_member()

이름 : 신신기
전화번호 : 010-9999-8989
이메일 : k@k.com
나이 : 22
고객등급(1~5) : 2
기타 정보 : etc
   **	신신기	010-9999-8989  	k@k.com        	 22	etc 님 데이터 저장완료


## 2번 기능. 전체조회

In [87]:
# 2. 전체조회 함수
def select_all():
    pass

In [101]:
# 위 test
select_all()

                                고객 정보                                 
----------------------------------------------------------------------
GRADE	이름 	    전화          메일       	나이 	기타
   **	신길동	010-8888-8888  	s@s.com        	 33	etc
*****	홍길동	010-7878-7878  	h@h.co.kr      	 41	글쎄
*****	윤길동	010-7898-8888  	a@a.com        	 43	etc
*****	신길동	010-9999-9999  	h@h.co.kr      	 33	열심히


## 3번 기능. 이름찾기

In [90]:
# 3. 이름찾기 함수        
def select_name():
    pass

In [102]:
# 위 함수 test
select_name()

검색할 이름을 입력하세요 신길동
   **	신길동	010-8888-8888  	s@s.com        	 33	etc
*****	신길동	010-9999-9999  	h@h.co.kr      	 33	열심히
이상 2명 조회됨


## 4번 기능. 메일로 삭제

In [98]:
# 4. 메일로 삭제 함수
def delete_email():
    pass

In [103]:
# 위 함수 test
delete_email()

삭제할 회원의 이메일을 입력하세요 h@h.co.kr
요청하신 메일 2행을 삭제하였습니다


## 5. 데이터베이스에서 조회한 정보를 CSV 파일로 내보내기

In [109]:
# 5. 데이터베이스에서 조회한 정보를 CSV 파일로 저장하는 함수
def save_csv():
    'sql 실행 결과를 튜플 리스트로 받아와서 db결과 title(description)과 튜플을 csv에 출력'
    pass

In [121]:
# 5. 데이터베이스에서 조회한 정보를 CSV 파일로 저장하는 함수
def save_csv():
    'sql 실행 결과를 딕셔너리 리스트로 받아와 csv에 출력'
    pass

In [122]:
save_csv()

파일명을 입력하세요(csv 확장자 포함)abc.csv
데이터 내보내기 예외 : 'Member' object has no attribute 'keys'
